# COMP4030 - Data Science and Machine Learning - Coursework 2

## Data collection and pre-processing
At this stage you can toggle which gestures you want to include in the training and testing by adding or removing elements from this functions return array.

In [ ]:
def get_gestures():
      return ['circle', 'go', 'come', 'wave'] # 

In [ ]:
# Utility functions - Ashley Hunt - psyah10
import os

def num_gestures():
      return len(get_gestures())

def get_columns():
    return ['time', 'accel_x', 'accel_y', 'accel_z', 'accel_abs']

def get_gesture_csvs(gesture_dir):
      if not os.path.exists(gesture_dir):
            os.makedirs(gesture_dir)
      return [file for file in os.listdir(gesture_dir) if file.endswith('.csv')]

### Data Importing
Next we read the raw data from the CSV files and place the data inside a pandas DataFrame. 

In [ ]:
# Data importing functions - Ashley Hunt - psyah10
import pandas as pd

## Simple function to read a csv file and return a dataframe
def get_df(path, trim=True):
      if not path.endswith('.csv'):
            return []
      
      raw_data = pd.read_csv(path)
      raw_data.columns = get_columns()

      df = pd.DataFrame(raw_data, columns=raw_data.columns)

      return trim_recording(df) if trim else df

## Function to trim the recording to the first and last significant movement
def trim_recording(df, window_size = 20, threshold = 0.3, padding=90):

      df['rolling_max'] = df['accel_abs'].rolling(window=window_size, min_periods=1).mean()

      start_cut = df[df['rolling_max'] >= threshold].index.min()
      if pd.notna(start_cut):
            cut_index = max(start_cut - padding, 0)
            df = df.loc[cut_index:]
      
      end_cut = df[df['rolling_max'] >= threshold].index.max()
      if pd.notna(end_cut):
            end_cut_index = min(end_cut + padding, len(df) - 1) 
            df = df.loc[:end_cut_index]

      df = df.drop('rolling_max', axis=1)
      return df

## Function to get all the data from the files in the data folder
def get_data_from_files(trim=True, test=False):
      sub_folder = 'test' if test else 'train'
      
      dfs = []
      gestures = get_gestures() + (['unknown'] if test else [])
      
      for gesture in gestures:
            folder_path = f'data/{sub_folder}/{gesture}'
            files_in_folder = get_gesture_csvs(folder_path)
            
            if len(files_in_folder) == 0:
                  continue
            
            for file_index, file_name in enumerate(files_in_folder):
                  file_path = os.path.join(folder_path, file_name)
                  df = get_df(file_path, trim)
                  if len(df) == 0:
                        continue

                  df['file_number'] = int(file_index)
                  df['gesture'] = str(gesture)

                  dfs.append(df)
                  
      df = pd.concat(dfs, ignore_index=True) if len(dfs) > 1 else dfs[0]
      df.set_index(['gesture', 'file_number'], inplace=True)
      df.sort_index(inplace=True)
      return df

In [ ]:
# Data importing - Ashley Hunt - psyah10

df = get_data_from_files(trim=False, test=False)
df

### Data Balancing
In order to ensure a balanced data set we find the minimum number of file contributions for each gesture and only take that number of files for every gesture.

In [ ]:
# Balance distribution functions - Ashley Hunt - psyah10

def balance_files(df):
      for gesture in df.index.get_level_values('gesture').unique():
            num_files = len(df.loc[(gesture), :].index.unique())
            print(f'{gesture} has {num_files} files (approx. {num_files * 8} gestures)')
            
      min_num_files = df.reset_index().groupby('gesture')['file_number'].nunique().min()
      print(f'\nAverage number of files per gesture: {min_num_files}')
      
      unique_file_numbers = df.index.get_level_values('file_number').unique()
      if len(unique_file_numbers) > min_num_files:
            remove = unique_file_numbers[min_num_files]
            balanced_df = df[df.index.get_level_values('file_number') < remove]
            print(f'\nRemoved from dataset where file_number >= {remove}')
      else :
            balanced_df = df
      
      return balanced_df

In [ ]:
# Balance distribution - Ashley Hunt - psyah10
df = balance_files(df)

### Unseen Test data
Alongside our labelled data sets we also process additional, unlabelled data that will not be seen my the model as it trains and will be used for validation only.

In [ ]:
# Test data importing - Ashley Hunt - psyah10

test_df = get_data_from_files(trim=False, test=True)
test_df = balance_files(test_df)

### Data visualisation and exploration
Next we visualise our raw data for exploratory analysis

In [ ]:
# Visualising data functions - Ashley Hunt - psyah10
import matplotlib.pyplot as plt

def visualise_df(dataframe, columns, files_per_gesture=2):
    
    second_level_name = dataframe.index.get_level_values(1).name
    num_gestures = dataframe.index.get_level_values('gesture').unique().size
    files = min(len(dataframe.index.get_level_values(second_level_name).unique()), files_per_gesture)
    
    fig, axs = plt.subplots(num_gestures, files, figsize=(14, num_gestures * 2))

    for (gesture, file_number), group in dataframe.loc[(slice(None), range(0,files)), :].groupby(level=['gesture', second_level_name]):

            try:
                indx = get_gestures().index(gesture)
            except ValueError:
                indx = num_gestures - 1
                
            ax =  axs[indx, file_number] if files > 1 else axs[indx]

            for col in columns:
                ax.plot(range(0, len(group)), group[col], label=col)
                ax.set_title("{gesture} {file_number}".format(gesture=gesture, file_number=file_number))
                ax.set_xlabel('Index')
                ax.set_ylabel('Acceleration')
                ax.legend()
            
    plt.tight_layout()
    plt.show()
    
def compare_dfs(df1, df2, visualise_n=3, new_label=''):
    
    num_rows = len(df1.index.get_level_values('gesture').unique()) * visualise_n
    
    fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(14, 2 * num_rows))

    for gn, (name, group) in enumerate(df1.groupby(level='gesture')):
        
        second_level_name = df1.index.get_level_values(1).name
        
        first_n_gestures = group.index.get_level_values(second_level_name).unique()[:visualise_n]
        data = group[group.index.get_level_values(second_level_name).isin(first_n_gestures)]
        
        for idx, (n, g_data) in enumerate(data.groupby(level=second_level_name)):
                
                row = idx + gn * visualise_n
                ax = axes[row, 0]
                for col in get_columns()[1:4]:
                    ax.plot(range(len(g_data)), g_data[col], label=col)
                ax.set_title(f"{name} {n}")
                ax.legend(loc='lower left')

                ax = axes[row, 1]
                filtered_data = df2.loc[(name, n)]
                for col in get_columns()[1:4]:
                    ax.plot(range(len(filtered_data)), filtered_data[col], label=col)
                ax.set_title(f'{name} {n} - After {new_label}')
                ax.legend(loc='lower left')

    plt.tight_layout()
    plt.show()

In [ ]:
visualise_df(df, ['accel_x', 'accel_y', 'accel_z'], files_per_gesture=2)

### EWMA Filtering
To eradicate spikes in the data, we apply exponentially weighted moving average (EWMA)

In [ ]:
# Apply exponential moving average - Ashley Hunt - psyah10
ewm_filtered_df = df.ewm(span=10, adjust=False).mean()
compare_dfs(df, ewm_filtered_df, visualise_n=2, new_label="EWMA filter")

In [ ]:
df = ewm_filtered_df

In [ ]:
ewm_filtered_test_df = test_df.ewm(span=10, adjust=False).mean()
compare_dfs(test_df, ewm_filtered_test_df, visualise_n=1, new_label="EWMA filter")

In [ ]:
test_df = ewm_filtered_test_df

### Low-Pass Filtering
Next we apply a low-pass filter to reduce noise from the data and make our model more robust. First we visualise our data and using interactive widgets we visually evaluate the performance of different parameter settings for apply the filter.

In [ ]:
# Low-pass filtering functions - Ashley Hunt - psyah10
from scipy.signal import butter, filtfilt

def butter_lowpass_filter(data, cutoff_freq, fs, order=5):
    nyquist_freq = 0.5 * fs
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

def apply_filter(df, cutoff_freq=2, fs=50, order=5):
    new_df = df.copy()
    for column in get_columns()[1:]:
        new_df[column] = butter_lowpass_filter(new_df[column], cutoff_freq, fs, order)
    return new_df

In [ ]:
# Interactive filter parameter tuning - Ashley Hunt - psyah10
%matplotlib inline
import ipywidgets as widgets
from IPython.display import display

cutoff_frequency_slider = widgets.IntSlider(value=3, min=1, max=15, step=1, description='cutoff_frequency')
sampling_rate_slider = widgets.IntSlider(value=150, min=10, max=150, step=1, description='sampling_rate')
filter_order_slider = widgets.IntSlider(value=3, min=1, max=15, step=1, description='filter_order')

def update_signal(cutoff_frequency, sampling_rate, filter_order):
    filtered_df = apply_filter(df, cutoff_frequency, sampling_rate, filter_order)
    compare_dfs(df, filtered_df, visualise_n=1)

interactive_plot = widgets.interactive(update_signal, cutoff_frequency=cutoff_frequency_slider, sampling_rate=sampling_rate_slider, filter_order=filter_order_slider)
display(interactive_plot)


Using the best parameters we then apply these settings to our data

In [ ]:
# Filter parameter tuning - Ashley Hunt - psyah10

cutoff_frequency = 3  # Cutoff frequency in Hz - Higher = less smoothing
sampling_rate = 150  # Sampling rate in Hz - Higher = more smoothing
filter_order = 5  # Filter order - Higher = less smoothing

df = apply_filter(df, cutoff_frequency, sampling_rate, filter_order)
test_df = apply_filter(test_df, cutoff_frequency, sampling_rate, filter_order)

### Gesture segmentation
Now we split the data from distinct files into distinct gestures. We do this by using the natural peaks and troughs in absolute acceleration.

In [ ]:
# Spltting gestures functions - Ashley Hunt - psyah10
from scipy.signal import find_peaks

def split_file_to_gestures(df, threshold, padding, height, distance, width, prominence):
      gesture_data = []
      charts = []
      for (gesture, file_number), group in df.groupby(level=['gesture', 'file_number']):
            
            charts.append(gesture) ## Limit to 2 charts per gesture
            
            group.reset_index(drop=True, inplace=True)
            group.drop(['time'], axis=1, inplace=True)
            
            peaks, peak_info = find_peaks(group['accel_abs'], height=height, distance=distance, width=width, prominence=prominence)
            
            if(len(peaks) < 6 or len(peaks) > 10):
                  print(f"Incorrect peaks in {gesture} #{file_number} ({len(peaks)} peaks)")
                  plt.plot(group['accel_abs'])
                  plt.plot(peaks, group['accel_abs'][peaks], "x")
                  plt.title(f'{gesture} {file_number}')
                  plt.show()
            
            for peak in peaks:
                  
                  below_target = group.loc[:peak]
                  start_index = below_target[below_target['accel_abs'] < threshold].last_valid_index() #or 0
                  start_index = max(0, start_index - padding)
                  
                  above_target = group.loc[peak + 1:]
                  end_index = above_target[above_target['accel_abs'] < threshold].first_valid_index() #or len(group) - 1
                  end_index = min(len(group), end_index + padding)
                  
                  data = group.loc[start_index:end_index].copy()
                  
                  data['gesture_number'] = len(gesture_data)
                  data['gesture'] = gesture
                  gesture_data.append(data)
      
      gesture_df = pd.concat(gesture_data, ignore_index=True)
      gesture_df.set_index(['gesture', 'gesture_number'], inplace=True)
      return gesture_df

In [ ]:
# Spltting gestures - Ashley Hunt - psyah10

# Parameters for splitting gestures, we adjust these to see which ones resulted in the best splitting

threshold=4
padding=15
height=3
distance=20
width=30
prominence=0

gesture_df = split_file_to_gestures(df, threshold, padding, height, distance, width, prominence)
test_gesture_df = split_file_to_gestures(test_df, threshold, padding, height, distance, width, prominence)

Now we visualise these gestures.

In [ ]:
# Visualising gesture functions - Ashley Hunt - psyah10

def visualise_gestures(dataframe, n):
      
      num_gestures = dataframe.index.get_level_values('gesture').unique().size
      
      charts = []
      plt.figure(figsize=(n * num_gestures, 10))
      for (gesture, gesture_number), group in dataframe.groupby(level=['gesture', 'gesture_number']):
            
            charts.append(gesture) ## Limit charts per gesture
            if(charts.count(gesture) > n):
                  continue
            
            gesture_i = len(pd.Series(charts).unique()) - 1
            plt.subplot(num_gestures, n, (n * gesture_i ) + charts.count(gesture) )
            
            group.reset_index(drop=True, inplace=True)
            for col in get_columns()[1:4]:
                  plt.plot(range(len(group)), group[col], label=col)
                  plt.title("{gesture} {gesture_number}".format(gesture=gesture, gesture_number=gesture_number))
                  plt.legend()
                  # plt.ylim(0, 1)
      plt.subplots_adjust(hspace=0.4)
      plt.show()

In [ ]:
# Visualising data - Ashley Hunt - psyah10

visualise_gestures(gesture_df, 8)

In [ ]:
# Visualising test data - Ashley Hunt - psyah10

visualise_gestures(test_gesture_df, 8)

If, due to slicing errors, there are duplicate gestures we will remove them

In [ ]:
# Removing duplicate functions - Ashley Hunt - psyah10

def remove_duplicate_gestures(dataframe):
      data = dataframe.copy()
      
      data_columns = get_columns()[1:]
      group_columns = ['gesture', 'gesture_number']

      grouped = data.groupby(group_columns)

      group_representations = {}
      duplicates_found = False
      for name, group in grouped:
            group_tuple = tuple(group.sort_values(by=data_columns)[data_columns].itertuples(index=False, name=None))
            
            if group_tuple in group_representations:
                  duplicates_found = True
                  data = data.drop(name)
                  print("Removing identical gestures", name, "and", group_representations[group_tuple])
            group_representations[group_tuple] = name
            
      if not duplicates_found:
            print("No duplicate gestures found")
      return data

In [ ]:
# Removing duplicate slices - Ashley Hunt - psyah10

gesture_df = remove_duplicate_gestures(gesture_df)
test_gesture_df = remove_duplicate_gestures(test_gesture_df)

### Fourier Transformation & Filtering
Next we apply a Fourier transformation on the data and filter out the frequencies in the data that are not adding useful information. This is achieved using trial and error through visualisation of the wave before and after transformation.

In [ ]:
import numpy as np

def compute_fft(data, sample_rate):
      fft_data = np.fft.fft(data)
      fft_magnitude = np.abs(fft_data)
      fft_frequency = np.fft.fftfreq(len(data), d=1/sample_rate)
      return fft_frequency, fft_magnitude

def filter_low_magnitude(frequencies, magnitudes):
      filtered_frequencies = []
      filtered_magnitudes = []
      
      for freq, mag in zip(frequencies, magnitudes):
            if mag >= 500 and freq >= 0:
                  filtered_frequencies.append(freq)
                  filtered_magnitudes.append(mag)

      filtered_frequencies = np.array(filtered_frequencies)
      filtered_magnitudes = np.array(filtered_magnitudes)
      
      return filtered_frequencies, filtered_magnitudes

def plot_frequency_spectrums(data, sample_rate):
      plt.figure(figsize=(12, 6))
      for col in get_columns()[1:4]:
            freq, mag = compute_fft(data[col], sample_rate)
            filtered_freq, filtered_mag = filter_low_magnitude(freq, mag)
            plt.plot(filtered_freq, filtered_mag, label=col)
            plt.title('Frequency Spectrum')
            plt.xlabel('Frequency (Hz)')
            plt.xlim(-0.1)
            plt.ylabel('Magnitude')
            plt.legend()
      plt.show()
      
plot_frequency_spectrums(gesture_df, 60)

In [ ]:
# Fourier Transformation functions - Ashley Hunt - psyah10
import numpy as np

def fft_filter(data, sample_rate, cutoff_freq_low, cutoff_freq_high):
    fft_data = np.fft.fft(data)
    freqs = np.fft.fftfreq(len(data), 1/sample_rate)

    mask = (freqs > cutoff_freq_low) & (freqs < cutoff_freq_high)
    fft_data[~mask] = 0

    filtered_signal = np.fft.ifft(fft_data).real
    return filtered_signal

def apply_fft_filter(data, sample_rate, cutoff_freq_low, cutoff_freq_high):
    new_df = data.copy()
    for column in get_columns()[1:4]:
        new_df[column] = fft_filter(new_df[column], sample_rate, cutoff_freq_low, cutoff_freq_high)
    return new_df

In [ ]:
# Fourier Transformation - Ashley Hunt - psyah10
 
sample_rate = 60  # Sampling rate (Hz)
cutoff_low = 0  # Low cutoff frequency (Hz)
cutoff_high = 1.5  # High cutoff frequency (Hz)

fft_filtered_gestures_df = apply_fft_filter(gesture_df, sample_rate, cutoff_low, cutoff_high)
fft_test_filtered_gestures_df = apply_fft_filter(test_gesture_df, sample_rate, cutoff_low, cutoff_high)

compare_dfs(gesture_df, fft_filtered_gestures_df, visualise_n=1, new_label="FFT Filtering")

In [ ]:
# Visualise the FFT filter alongside the original data (test data)
compare_dfs(test_gesture_df, fft_test_filtered_gestures_df, 3)

### Data Normalisation

At this point we also normalise our data using a MinMaxScaler, ensuring that all of our data lies between 0 and 1. This will ensure that no single field plays a more important role than it should.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() ## StandardScaler() or MinMaxScaler()

gesture_df[gesture_df.columns] = scaler.fit_transform(gesture_df[gesture_df.columns])
test_gesture_df[test_gesture_df.columns] = scaler.fit_transform(test_gesture_df[test_gesture_df.columns])

## Feature Extraction
Next we can begin to extract our features for the models. These functions split the data into a fixed number of segments with some overlap and then take some key statistical values from these slices.

In [ ]:
# Feature extraction functions - Ashley Hunt - psyah10

def extract_segments(df, num_segments, overlap_fraction):
    total_rows = len(df)
    segment_size = total_rows // num_segments
    overlap_size = int(segment_size * overlap_fraction)
    
    segments = []
    
    for i in range(num_segments):
        start_idx = i * segment_size
        if i > 0:
            start_idx -= overlap_size
        
        if i == num_segments - 1:
            end_idx = total_rows
        else:
            end_idx = start_idx + segment_size + overlap_size

        segments.append(df.iloc[start_idx:end_idx])
    
    return segments

def extract_features_from_df(df, feature_functions, num_segments = 4, overlap_fraction = 0.1):
      results = []
      
      for (gesture, gesture_number), group in df.groupby(level=['gesture', 'gesture_number']):
            
            group.reset_index(drop=True, inplace=True)

            result_dict = {}
            segments = extract_segments(group, num_segments, overlap_fraction)
            result_dict['gesture'] = gesture
            for n, segment in enumerate(segments):
                  for col in get_columns()[1:4]:
                        for f in feature_functions:
                              result_dict[f'{col}_{f}_{n+1}'] = segment[col].agg(f)

            results.append(result_dict)

      return pd.DataFrame(results)

In [ ]:
# Feature extraction - Ashley Hunt - psyah10

# Function options 'mean', 'min', 'max', 'median', 'std', 'skew', 'kurtosis', 'quantile'
feature_functions = ['mean', 'min', 'max', 'median', 'std', 'skew', 'kurtosis', 'quantile'] 
num_segments = 8
overlap_fraction = 0.5

# If using FFT filtered data
extracted_features = extract_features_from_df(fft_filtered_gestures_df, feature_functions, num_segments, overlap_fraction)
extracted_test_features = extract_features_from_df(fft_test_filtered_gestures_df, feature_functions, num_segments, overlap_fraction)

# If NOT using FFT filtered data
# extracted_features = extract_features_from_df(gesture_df, feature_functions, num_segments, overlap_fraction)
# extracted_test_features = extract_features_from_df(test_gesture_df, feature_functions, num_segments, overlap_fraction)

In [ ]:
extracted_features.dropna(how='any', inplace=True)
extracted_test_features.dropna(how='any', inplace=True)

extracted_features


In [ ]:
from itertools import cycle
# For each 'uknown' gesture in extracted_test_features replace with gestures from get_gestures() in a cycle

repeats_in_tests = 2
repeated = [element for element in get_gestures() for _ in range(repeats_in_tests)]
unknown_indices = extracted_test_features.index[extracted_test_features['gesture'] == 'unknown'].tolist()

gesture_cycle = cycle(repeated)
replacements = [next(gesture_cycle) for _ in range(len(unknown_indices))]

for index, replacement in zip(unknown_indices, replacements):
    extracted_test_features.at[index, 'gesture'] = replacement

## Feature Selection
Now we visulaise the extracted features to help us determine which features are most useful in classification.

In [ ]:
# Visualisting Feature extraction functions - Ashley Hunt - psyah10

def show_trends(dataframe):
      for f in feature_functions:
            plt.figure(figsize=(15, 3))
            for col in get_columns()[1:4]:
                  plt.subplot(1, 3, get_columns().index(col))
                  plt.title(col)
                  for g in get_gestures():
                        points = []
                        for s in range(1, num_segments + 1):
                              points.append( dataframe.loc[dataframe["gesture"] == g][f'{col}_{f}_{s}'].mean())
                        plt.plot(range(1, num_segments + 1), points, marker='o', linestyle='-', label=g)
                        plt.xlabel('Segment')
                        plt.ylabel(f)
                        plt.legend()
                        plt.xticks(range(1, num_segments + 1))
            plt.suptitle(f"{f} by segment")
            plt.tight_layout()
            plt.show()

### Feature visualisation

In [ ]:
# Visualisting Feature extraction for feature selection - Ashley Hunt - psyah10

# show_trends(extracted_features)

In [ ]:
# Visualisting individual feature extraction for selection - Ashley Hunt - psyah10

def show_trend(dataframe, function, charts_per_gesture=4):
      
      for g in get_gestures():
            if len(dataframe.loc[dataframe["gesture"] == g]) == 0:
                  continue
            plt.figure(figsize=(charts_per_gesture * 4, 3))
            plt.title(f"{charts_per_gesture} {g}'s {function}")
            for graphs in range(0, charts_per_gesture):
                  plt.subplot(1, charts_per_gesture, graphs + 1)
                  
                  data = dataframe.loc[dataframe["gesture"] == g].iloc[graphs]
                  for col in get_columns()[1:4]:
                        points = []
                        for s in range(1, num_segments + 1):
                              points.append(data[f'{col}_{function}_{s}'])
                        plt.plot(range(1, num_segments + 1), points, marker='o', linestyle='-', label=col)
                  plt.legend()
                  plt.xticks(range(1, num_segments + 1))
            plt.tight_layout()
            plt.show()

In [ ]:
# Visualisting Feature extraction for feature selection - Ashley Hunt - psyah10

# for f in feature_functions:
#       show_trend(extracted_features, f, 2)

### Rebalance data
Once again we reblance our data, this time on the total number of observations for each gesture.

In [ ]:
# Balance data functions - Ashley Hunt - psyah10

def rebalance_data(dataframe):
      # Find out the minimum number of gestures in a category
      min_gestures = dataframe['gesture'].value_counts().min()

      # Take only the top x gestures
      return dataframe.groupby('gesture').head(min_gestures)


In [ ]:
# Re-balance data - Ashley Hunt - psyah10

extracted_features = rebalance_data(extracted_features)

In [ ]:
# Principle Component Analysis & Extraction functions - Ashley Hunt - psyah10

from sklearn.decomposition import PCA
import seaborn as sns

def extract_principle_components(dataframe, components=2):
      
      data = dataframe.reset_index()
      gestures = data['gesture']
      data = data.loc[:, (data.columns != 'gesture') & (data.columns != 'index')]

      pca = PCA(n_components=components)
      principal_components = pca.fit_transform(data)
      cols = [f'principal component {i}' for i in range(1, components + 1)]
      principle_df = pd.DataFrame(data = principal_components, columns = cols)
      principle_df['gesture'] = gestures

      return principle_df

def visualise_principle_components(dataframe):
      cols = len(dataframe.columns) - 1
      plt.figure(figsize=(cols * 3, cols * 3))
      sns.pairplot(dataframe, hue="gesture", palette="deep")
      plt.show()

### Principle Component Analysis

We then utilise PCA to do automatic feature selection. This is easy to extract and does not rely on subjective human input.

In [ ]:
# Principle Component Analysis & Extraction - Ashley Hunt - psyah10

NUM_PCA_COMPONENTS = 2

principle_features = extract_principle_components(extracted_features, components=NUM_PCA_COMPONENTS)
test_principle_features = extract_principle_components(extracted_test_features, components=NUM_PCA_COMPONENTS)

In [ ]:
# Visualising Principle Component Analysis - Ashley Hunt - psyah10
visualise_principle_components(principle_features)

In [ ]:
# Visualising Principle Component Analysis - Ashley Hunt - psyah10
visualise_principle_components(test_principle_features)

## Model Training
First we prepare out label encoder

In [ ]:
# Label encoding functions - Ashley Hunt - psyah10
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

def encode_labels(labels):
    return label_encoder.fit_transform(labels)

def decode_labels(encoded_labels):
    return label_encoder.inverse_transform(encoded_labels)

Then we extract the data that we need

In [ ]:
# Data splitting - Ashley Hunt - psyah10
from sklearn.model_selection import train_test_split

PRINCIPLE = True

if PRINCIPLE:
      X = principle_features.loc[:, principle_features.columns != 'gesture']
      Y = encode_labels(principle_features['gesture'])
      UNSEEN_FEATURES = test_principle_features.loc[:, test_principle_features.columns != 'gesture']
      UNSEEN_LABELS = encode_labels(test_principle_features['gesture'])
else:
      X = extracted_features.loc[:, extracted_features.columns != 'gesture']
      Y = encode_labels(extracted_features['gesture'])
      UNSEEN_FEATURES = extracted_test_features.loc[:, extracted_test_features.columns != 'gesture']
      UNSEEN_LABELS = encode_labels(extracted_test_features['gesture'])

train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size=0.3, random_state=42)

### K-means Clustering
As the first, most basic, option for classification we consider a K-means clustering algorithm.

In [ ]:
# K-means clustering functions - Ashley Hunt - psyah10

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

k = num_gestures()

kmeans_model = KMeans(n_clusters=k, random_state=42, n_init='auto')
kmeans_model.fit(train_features)

In [ ]:
# K-means clustering visualisation - Ashley Hunt - psyah10

def visualise_kmeans(dataframe, model):
      kmeans_data = dataframe.copy()
      kmeans_data['Cluster'] = model.fit_predict(dataframe)
      kmeans_data['Gesture'] = decode_labels(train_labels)

      sns.scatterplot( data = kmeans_data, x='principal component 1', y='principal component 2', hue='Gesture', palette='viridis', style='Cluster', s=100)
      plt.scatter(model.cluster_centers_[:, 0], model.cluster_centers_[:, 1], c='red', s=100, alpha=0.5, label='Centroids')
      plt.title('K-means Clustering on PCA-Reduced Data')
      plt.xlabel('principal component 1')
      plt.ylabel('principal component 2')
      plt.legend()
      plt.show()

In [ ]:
visualise_kmeans(train_features, kmeans_model)

In [ ]:
# Testing/Validation display functions - Ashley Hunt - psyah10

from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def show_confusion_matrix(actual, predictions):
      
      fig, ax = plt.subplots(figsize=(8, 6))
      
      labels = decode_labels(unique_labels(actual))
      cm_df = pd.DataFrame(confusion_matrix(actual, predictions), index=labels, columns=labels)

      # plt.figure(figsize=(7, 5))
      sns.heatmap(cm_df, annot=True, fmt="d", cmap='Blues', cbar=True, square=True, linewidth=.5, ax=ax)
      
      # Loop over the data dimensions and outline the diagonal cells
      for i in range(cm_df.shape[0]):
            for j in range(cm_df.shape[1]):
                  if i == j: 
                        rect = plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='green', lw=1)
                        ax.add_patch(rect)
      
      plt.title('Confusion Matrix')
      plt.ylabel('Actual', labelpad=20)
      plt.xlabel('Predicted', labelpad=20)
      plt.tight_layout()
      plt.show()
      
def show_classification_report(actual, predictions):
      print(f"Accuracy: {accuracy_score(actual, predictions)}")
      print(classification_report(actual, predictions))

In [ ]:
# K-means Evaluation - Ashley Hunt - psyah10
kmeans_predictions = kmeans_model.predict(test_features)
print('Silhouette Score:', silhouette_score(test_features, kmeans_predictions))
show_classification_report(test_labels, kmeans_predictions)

In [ ]:
# K-means Validation - Ashley Hunt - psyah10

kmeans_prediction = kmeans_model.predict(UNSEEN_FEATURES)

show_classification_report(UNSEEN_LABELS, kmeans_prediction)
show_confusion_matrix(UNSEEN_LABELS, kmeans_prediction)

### Random Forest

In [ ]:
# Random Forest Training - Ashley Hunt - psyah10
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(n_estimators=100, random_state=42)
rfc_model.fit(train_features, train_labels)

In [ ]:
# Random Forest Evaluation - Ashley Hunt - psyah10

rfc_prediction = rfc_model.predict(test_features)

show_classification_report(test_labels, rfc_prediction)

In [ ]:
# Random Forest Hypertuning  Ashley Hunt - psyah10
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }
rf = RandomForestClassifier(random_state=42)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, error_score='raise')
rf_random.fit(train_features, train_labels)

In [ ]:
# Random Forest Hypertuning  Ashley Hunt - psyah10
from sklearn.metrics import balanced_accuracy_score

rf_best_params = rf_random.best_params_
rf_best_estimator = rf_random.best_estimator_

rf_predictions = rf_best_estimator.predict(test_features)
balanced_accuracy = balanced_accuracy_score(test_labels, rf_predictions)

print("Best Parameters:", rf_best_params)
print("Balanced Accuracy:", balanced_accuracy)

In [ ]:
# Random Forest Validation - Ashley Hunt - psyah10
rfc_unseen_feature_prediction = rf_best_estimator.predict(UNSEEN_FEATURES)

show_classification_report(UNSEEN_LABELS, rfc_unseen_feature_prediction)
show_confusion_matrix(UNSEEN_LABELS, rfc_unseen_feature_prediction)

### Support Vector Machines 

In [ ]:
# Support Vector Machines Training - Ashley Hunt - psyah10
from sklearn import svm

svm_model = svm.SVC(kernel='linear')
svm_model.fit(train_features, train_labels)

In [ ]:
# Support Vector Machines Evaluation - Ashley Hunt - psyah10
svm_prediction = svm_model.predict(test_features)

show_classification_report(test_labels, svm_prediction)

In [ ]:
# Support Vector Machines Validation - Ashley Hunt - psyah10

svm_unseen_feature_prediction = svm_model.predict(UNSEEN_FEATURES)

show_classification_report(UNSEEN_LABELS, svm_unseen_feature_prediction)
show_confusion_matrix(UNSEEN_LABELS, svm_unseen_feature_prediction)

### XGBoost

In [ ]:
# XGB - Amit Kumar
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_classifier = XGBClassifier(random_state=42)

param_grid = {
    'n_estimators': [300],
    'max_depth': [20],
    'learning_rate': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [1]
}

#param_grid = {
    #'n_estimators': [50, 100, 150],
    #'max_depth': [3, 5, 7],
    #'learning_rate': [0.1, 0.01, 0.001],
    #'subsample': [0.8, 0.9, 1.0],
    #'colsample_bytree': [0.8, 0.9, 1.0]
#}

knn_grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, scoring='accuracy')
knn_grid_search.fit(train_features, train_labels)

best_params = knn_grid_search.best_params_
best_estimator = knn_grid_search.best_estimator_

rf_predictions = best_estimator.predict(test_features)

print("Best Parameters:", best_params)
print("Accuracy:", accuracy_score(test_labels, rf_predictions))

In [ ]:
# XGB Validation - Ashley Hunt - psyah10

xgb_unseen_feature_prediction = best_estimator.predict(UNSEEN_FEATURES)

show_classification_report(UNSEEN_LABELS, xgb_unseen_feature_prediction)
show_confusion_matrix(UNSEEN_LABELS, xgb_unseen_feature_prediction)

### Long Short-Term Memory layer (LSTM)

In [ ]:
# LSTM - Amit Kumar
from keras.models import Sequential
from keras.layers import LSTM, Dense

data = df.copy().reset_index()

# Encode the gesture labels
label_encoder = LabelEncoder()
data['gesture_encoded'] = label_encoder.fit_transform(data['gesture'])

# Normalize the features
scaler = StandardScaler()
data[['accel_x', 'accel_y', 'accel_z', 'accel_abs']] = scaler.fit_transform(data[['accel_x', 'accel_y', 'accel_z', 'accel_abs']])

# Define the time steps
time_steps = 10

# Prepare the data for LSTM
def prepare_data_for_lstm(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:(i + time_steps)].values)
        y.append(data.iloc[i + time_steps]['gesture_encoded'])
    return np.array(X), np.array(y)

X, y = prepare_data_for_lstm(data[['accel_x', 'accel_y', 'accel_z', 'accel_abs', 'gesture_encoded']], time_steps)

# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(len(label_encoder.classes_), activation='softmax')) # Change activation to softmax for multi-class classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Change loss function for multi-class classification

# Train the model with validation data
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
_, test_accuracy = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_accuracy)

# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
# LSTM - Amit Kumar
test_data = test_df.copy().reset_index()

# Preprocess the new data
test_data['gesture_encoded'] = label_encoder.transform(test_data['gesture'])  # Use the same label encoder as before
test_data[['accel_x', 'accel_y', 'accel_z', 'accel_abs']] = scaler.transform(test_data[['accel_x', 'accel_y', 'accel_z', 'accel_abs']])
 
# Prepare the new data for evaluation
X_new, y_new = prepare_data_for_lstm(test_data[['accel_x', 'accel_y', 'accel_z', 'accel_abs', 'gesture_encoded']], time_steps)
 
y_pred_prob = model.predict(X_new)
y_pred = np.argmax(y_pred_prob, axis=1)
 
# Create a mask for the misclassified samples where the predicted label matches the input label
misclassified_mask = y_pred == y_new
 
# Invert the mask to get the indices of misclassified samples
misclassified_indices = np.logical_not(misclassified_mask)
 
# Decode the predicted labels back into gesture names
predicted_gestures = label_encoder.inverse_transform(y_pred)
 
# Display the gestures with their corresponding predicted labels and accuracy
print("Gesture\t\tPredicted Label")
print("=============================")
for i in range(len(predicted_gestures)):
    # If the sample is misclassified, mark it as such in the output
    if misclassified_indices[i]:
        print(f"{test_data.iloc[i]['gesture']}\t\t{predicted_gestures[i]} (Misclassified)")
    else:
        print(f"{test_data.iloc[i]['gesture']}\t\t{predicted_gestures[i]}")
 
# Calculate and display the overall accuracy
correct_predictions = np.sum(misclassified_mask)
total_predictions = len(y_new)
overall_accuracy = correct_predictions / total_predictions
print("\nOverall Accuracy on Test Data:", overall_accuracy)
 

### KNN

In [ ]:
# KNN - Shreeya Kumbhoje - psxsk19
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


# scaler = StandardScaler()
knn = KNeighborsClassifier()
 

pipeline = Pipeline([('knn', knn)])


param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11], 
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}
 

knn_grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)
knn_grid_search.fit(train_features, train_labels)
 

print("Best parameters:", knn_grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(knn_grid_search.best_score_))
 

knn_best_model = knn_grid_search.best_estimator_
knn_predictions = knn_best_model.predict(test_features)

show_classification_report(test_labels, knn_predictions)

In [ ]:
knn_unseen_test_pred = knn_best_model.predict(UNSEEN_FEATURES)

show_classification_report(UNSEEN_LABELS, knn_unseen_test_pred)
show_confusion_matrix(UNSEEN_LABELS, knn_unseen_test_pred)

### GRU

In [ ]:
# GRU Model - Shreeya - psxsk19

gesture_df.reset_index(inplace=True)
gesture_df

In [ ]:
#Shreeya Kumbhoje - psxsk19
import numpy as np
import tensorflow as tf
import random
import os


np.random.seed(38)
random.seed(38)
tf.random.set_seed(38)


os.environ['PYTHONHASHSEED'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
#Shreeya Kumbhoje - psxsk19

import numpy as np
import tensorflow as tf
import random
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Set seed for reproducibility
np.random.seed(38)
random.seed(38)
tf.random.set_seed(38)
os.environ['PYTHONHASHSEED'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

gesture_df.reset_index()


N = gesture_df['gesture'].nunique()


gesture_df['label'] = gesture_df['gesture'].astype('category').cat.codes


grouped = gesture_df.groupby(['gesture', 'gesture_number'])
sequences = [group[1][['accel_x', 'accel_y', 'accel_z', 'accel_abs']].values for group in grouped]
labels = [group[1]['label'].iloc[0] for group in grouped]


sequences_train, sequences_test, labels_train, labels_test = train_test_split(
    sequences, labels, test_size=0.2, random_state=42)


X_train = pad_sequences(sequences_train, padding='post', dtype='float32')
X_test = pad_sequences(sequences_test, padding='post', dtype='float32', maxlen=X_train.shape[1])


y_train = to_categorical(labels_train, num_classes=N)
y_test = to_categorical(labels_test, num_classes=N)


timesteps = X_train.shape[2]  
model = Sequential([
    GRU(64, input_shape=(timesteps, 4), return_sequences=True),
    GRU(32),
    Dense(N, activation='softmax')  
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=25, batch_size=10)


test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')